<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEmotionnegpos4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Mounted at /content/drive
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...

In [ ]:
df_train.head(1000)

,Tweet,Emotion
8535,michaelbolton yesmet jon last yr at an interes...,2
4075,will be attend cic parent orient tomorrowne to...,1
8194,at school again it bore and these peopl are pl...,1
10237,ellebella omg I am so sorri anyth i can do to ...,0
6785,wish i wa in chicago,2
...,...,...
12109,take care of yucki stuff,0
6758,alfiejg i do not think i am my sister refusn t...,2
12123,i cannot find my woodi and buzz toy but ive go...,2
11454,theoriginalteam ooh can i have some I am hungr...,0


In [ ]:
df_test.Emotion.unique()

array(['0', '2', '1', '3', 'emotion'], dtype=object)

In [ ]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [ ]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [ ]:
print(len(train_data_cat))
print(len(test_data_cat))

12428
12428


In [ ]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)


In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
def save_dict_to_file(dic):
    f = open("/content/drive/My Drive/InfluenceAnalysis/vocab1.txt",'w')
    for i in dic:
      f.write(str(i)+"\t")
      f.write(str(w2v_sg.wv.get_vector(i)))
      f.write("\n")
    f.close()
save_dict_to_file(vocab_sg)

In [ ]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [ ]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1

for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [ ]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
X_train = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train]
X_test = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0]*mul_factor),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0]),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0]),tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          tweet[i]=((tweet[i][0]*tweet[i][2]),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0]*mul_factor),tweet[i][1],tweet[i][2])
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='NN':
                tweet[z]=((tweet[z][0]+tweet[i][0]),tweet[z][1])
          if tweet[i][1][:2]=='RB':
            for z in range(max(i-2,0),min(i+3,l)):
              if tweet[z][1][:2]=='VB':
                tweet[z]=((tweet[z][0]+tweet[i][0]),tweet[z][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs = 40

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
24/24 [==============================] - 6s 242ms/step - loss: 1.2647 - accuracy: 0.5685
Epoch 2/40
24/24 [==============================] - 6s 244ms/step - loss: 1.0252 - accuracy: 0.5847
Epoch 3/40
24/24 [==============================] - 6s 244ms/step - loss: 0.9864 - accuracy: 0.5847
Epoch 4/40
24/24 [==============================] - 6s 244ms/step - loss: 0.9819 - accuracy: 0.5847
Epoch 5/40
24/24 [==============================] - 6s 239ms/step - loss: 0.9769 - accuracy: 0.5847
Epoch 6/40
24/24 [==============================] - 6s 242ms/step - loss: 0.9728 - accuracy: 0.5847
Epoch 7/40
24/24 [==============================] - 6s 239ms/step - loss: 0.9686 - accuracy: 0.5847
Epoch 8/40
24/24 [==============================] - 6s 242ms/step - loss: 0.9638 - accuracy: 0.5847
Epoch 9/40
24/24 [==============================] - 6s 240ms/step - loss: 0.9585 - accuracy: 0.5849
Epoch 10/40
24/24 [=====

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/40
24/24 [==============================] - 6s 239ms/step - loss: 1.2167 - accuracy: 0.5478
Epoch 2/40
24/24 [==============================] - 6s 245ms/step - loss: 0.9949 - accuracy: 0.5847
Epoch 3/40
24/24 [==============================] - 6s 248ms/step - loss: 0.9740 - accuracy: 0.5848
Epoch 4/40
24/24 [==============================] - 6s 243ms/step - loss: 0.9665 - accuracy: 0.5844
Epoch 5/40
24/24 [==============================] - 6s 241ms/step - loss: 0.9615 - accuracy: 0.5842
Epoch 6/40
24/24 [==============================] - 6s 239ms/step - loss: 0.9566 - accuracy: 0.5842
Epoch 7/40
24/24 [==============================] - 6s 239ms/step - loss: 0.9510 - accuracy: 0.5847
Epoch 8/40
24/24 [==============================] - 6s 243ms/step - loss: 0.9469 - accuracy: 0.5857
Epoch 9/40
24/24 [==============================] - 6s 242ms/step - loss: 0.9426 - accuracy: 0.5862
Epoch 10/40
24/24 [==============================] - 6s 236ms/step - loss: 0.9376 - accuracy: 0.5867

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/40
24/24 [==============================] - 7s 273ms/step - loss: 1.1181 - accuracy: 0.5297
Epoch 2/40
24/24 [==============================] - 6s 263ms/step - loss: 0.9919 - accuracy: 0.5847
Epoch 3/40
24/24 [==============================] - 6s 261ms/step - loss: 0.9866 - accuracy: 0.5847
Epoch 4/40
24/24 [==============================] - 6s 254ms/step - loss: 0.9860 - accuracy: 0.5847
Epoch 5/40
24/24 [==============================] - 6s 254ms/step - loss: 0.9858 - accuracy: 0.5848
Epoch 6/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9857 - accuracy: 0.5847
Epoch 7/40
24/24 [==============================] - 6s 256ms/step - loss: 0.9857 - accuracy: 0.5847
Epoch 8/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9856 - accuracy: 0.5848
Epoch 9/40
24/24 [==============================] - 6s 251ms/step - loss: 0.9858 - accuracy: 0.5848
Epoch 10/40
24/24 [==============================] - 6s 255ms/step - loss: 0.9856 - accuracy: 0.5847

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/40
24/24 [==============================] - 6s 239ms/step - loss: 1.2622 - accuracy: 0.5691
Epoch 2/40
24/24 [==============================] - 6s 243ms/step - loss: 1.0054 - accuracy: 0.5847
Epoch 3/40
24/24 [==============================] - 6s 241ms/step - loss: 0.9848 - accuracy: 0.5847
Epoch 4/40
24/24 [==============================] - 6s 238ms/step - loss: 0.9830 - accuracy: 0.5848
Epoch 5/40
24/24 [==============================] - 6s 239ms/step - loss: 0.9808 - accuracy: 0.5848
Epoch 6/40
24/24 [==============================] - 6s 237ms/step - loss: 0.9778 - accuracy: 0.5847
Epoch 7/40
24/24 [==============================] - 6s 238ms/step - loss: 0.9744 - accuracy: 0.5847
Epoch 8/40
24/24 [==============================] - 6s 236ms/step - loss: 0.9704 - accuracy: 0.5847
Epoch 9/40
24/24 [==============================] - 6s 244ms/step - loss: 0.9665 - accuracy: 0.5847
Epoch 10/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9631 - accuracy: 0.5847

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/40
24/24 [==============================] - 6s 244ms/step - loss: 1.2222 - accuracy: 0.5440
Epoch 2/40
24/24 [==============================] - 6s 246ms/step - loss: 1.0046 - accuracy: 0.5847
Epoch 3/40
24/24 [==============================] - 6s 248ms/step - loss: 0.9752 - accuracy: 0.5847
Epoch 4/40
24/24 [==============================] - 6s 247ms/step - loss: 0.9664 - accuracy: 0.5847
Epoch 5/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9587 - accuracy: 0.5847
Epoch 6/40
24/24 [==============================] - 6s 254ms/step - loss: 0.9534 - accuracy: 0.5847
Epoch 7/40
24/24 [==============================] - 6s 247ms/step - loss: 0.9490 - accuracy: 0.5855
Epoch 8/40
24/24 [==============================] - 6s 245ms/step - loss: 0.9445 - accuracy: 0.5863
Epoch 9/40
24/24 [==============================] - 6s 243ms/step - loss: 0.9402 - accuracy: 0.5881
Epoch 10/40
24/24 [==============================] - 6s 250ms/step - loss: 0.9359 - accuracy: 0.5899

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/40
24/24 [==============================] - 6s 258ms/step - loss: 1.1680 - accuracy: 0.5102
Epoch 2/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9998 - accuracy: 0.5842
Epoch 3/40
24/24 [==============================] - 6s 253ms/step - loss: 0.9889 - accuracy: 0.5846
Epoch 4/40
24/24 [==============================] - 6s 255ms/step - loss: 0.9868 - accuracy: 0.5847
Epoch 5/40
24/24 [==============================] - 6s 257ms/step - loss: 0.9857 - accuracy: 0.5848
Epoch 6/40
24/24 [==============================] - 6s 255ms/step - loss: 0.9854 - accuracy: 0.5848
Epoch 7/40
24/24 [==============================] - 6s 256ms/step - loss: 0.9853 - accuracy: 0.5849
Epoch 8/40
24/24 [==============================] - 6s 251ms/step - loss: 0.9853 - accuracy: 0.5848
Epoch 9/40
24/24 [==============================] - 6s 248ms/step - loss: 0.9853 - accuracy: 0.5848
Epoch 10/40
24/24 [==============================] - 6s 252ms/step - loss: 0.9853 - accuracy: 0.5848

KeyboardInterrupt: ignored

In [ ]:
epochs=40
Embedding_size=100

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

In [ ]:
model_sg1 = tf.keras.Sequential()
model_sg1.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg1.add(tf.keras.layers.LSTM(64))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg1.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg1.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg1.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

In [ ]:
print(result_table[9])